# Train a Q&A model
In this notebook we fine tune a transformer model on a dataset using the HuggingFace datasets and transformers library.

In [1]:
! pip3 install datasets transformers

     |████████████████████████████████| 290 kB 12.6 MB/s 
     |████████████████████████████████| 3.1 MB 51.1 MB/s 
     |████████████████████████████████| 59 kB 6.5 MB/s 
     |████████████████████████████████| 1.1 MB 40.0 MB/s 
     |████████████████████████████████| 132 kB 50.5 MB/s 
     |████████████████████████████████| 243 kB 52.3 MB/s 
     |████████████████████████████████| 3.3 MB 37.5 MB/s 
     |████████████████████████████████| 596 kB 40.9 MB/s 
     |████████████████████████████████| 895 kB 46.6 MB/s 
     |████████████████████████████████| 271 kB 52.5 MB/s 
     |████████████████████████████████| 160 kB 49.2 MB/s 
     |████████████████████████████████| 192 kB 51.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Training a question answering model

In this notebook, we will see how to fine-tune one of the 🤗 Transformers model to a question answering task, which is the task of extracting the answer to a question from a given context. We will see how to easily load a dataset for these kinds of tasks and use the Trainer API to fine-tune a model on it.

In [2]:
import transformers

print(transformers.__version__)

4.12.5


In [3]:
squad_v2 = True
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## Loading the dataset

In [4]:
from datasets import load_dataset, load_metric, DatasetDict

In [5]:
train, validation = load_dataset("squad_v2" if squad_v2 else "squad", split=['train[:10%]', 'validation']) 

Downloading:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/9.55M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/801k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
datasets = DatasetDict()

In [7]:
datasets["train"] = train
datasets["validation"] = validation

In [8]:
datasets["validation"][5]

{'answers': {'answer_start': [], 'text': []},
 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'id': '5ad39d53604f3c001a3fe8d1',
 'question': "Who gave their name to Normandy in the 1000's and 1100's",
 'title': 'Normans'}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).


In [9]:
from utils import *

In [10]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,56d2211ae7d4791d00902694,Frédéric_Chopin,"During 1824–28 Chopin spent his vacations away from Warsaw, at a number of locales.[n 4] In 1824 and 1825, at Szafarnia, he was a guest of Dominik Dziewanowski, the father of a schoolmate. Here for the first time he encountered Polish rural folk music. His letters home from Szafarnia (to which he gave the title ""The Szafarnia Courier""), written in a very modern and lively Polish, amused his family with their spoofing of the Warsaw newspapers and demonstrated the youngster's literary gift.",What was the title chopin gave of some spoof letters he wrote?,"{'text': ['The Szafarnia Courier'], 'answer_start': [314]}"
1,56bfe2a2a10cfb140055136a,Beyoncé,"Beyoncé and her mother introduced House of Deréon, a contemporary women's fashion line, in 2005. The concept is inspired by three generations of women in their family, the name paying tribute to Beyoncé's grandmother, Agnèz Deréon, a respected seamstress. According to Tina, the overall style of the line best reflects her and Beyoncé's taste and style. Beyoncé and her mother founded their family's company Beyond Productions, which provides the licensing and brand management for House of Deréon, and its junior collection, Deréon. House of Deréon pieces were exhibited in Destiny's Child's shows and tours, during their Destiny Fulfilled era. The collection features sportswear, denim offerings with fur, outerwear and accessories that include handbags and footwear, and are available at department and specialty stores across the US and Canada.",When did Beyonce and her mother start Dereon?,"{'text': ['2005'], 'answer_start': [91]}"
2,56d43da72ccc5a1400d830bd,Beyoncé,"Following the disbandment of Destiny's Child in June 2005, she released her second solo album, B'Day (2006), which contained hits ""Déjà Vu"", ""Irreplaceable"", and ""Beautiful Liar"". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-setting six Grammy Awards in 2010, including Song of the Year for ""Single Ladies (Put a Ring on It)"". Beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. Her critically acclaimed fifth studio album, Beyoncé (2013), was distinguished from previous releases by its experimental production and exploration of darker themes.",When did Destiny's Child end their group act?,"{'text': ['June 2005'], 'answer_start': [48]}"
3,56bf95eaa10cfb1400551195,Beyoncé,"Her first acting role of 2006 was in the comedy film The Pink Panther starring opposite Steve Martin, grossing $158.8 million at the box office worldwide. Her second film Dreamgirls, the film version of the 1981 Broadway musical loosely based on The Supremes, received acclaim from critics and grossed $154 million internationally. In it, she starred opposite Jennifer Hudson, Jamie Foxx, and Eddie Murphy playing a pop singer based on Diana Ross. To promote the film, Beyoncé released ""Listen"" as the lead single from the soundtrack album. In April 2007, Beyoncé embarked on The Beyoncé Experience, her first worldwide concert tour, visiting 97 venues and grossed over $24 million.[note 1] Beyoncé conducted pre-concert food donation drives during six major stops in conjunction with her pastor at St. John's and America's Second Harvest. At the same time, B'Day was re-released with five additional songs, including her duet with Shakira ""Beautiful Liar"".",What did Beyonce call her first concert tour?,"{'text': ['The Beyoncé Experience'], 'answer_start': [576]}"
4,56bfbda3a10cfb1400551297,Beyoncé,"The feminism and female em

## Preprocessing the training data

In [11]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [12]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [13]:
pad_on_right = tokenizer.padding_side == "right"

In [14]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [15]:
features = prepare_train_features(datasets["train"][:5], tokenizer, pad_on_right, max_length, doc_stride)

In [16]:
tokenized_datasets = datasets.map(lambda x: prepare_train_features(x, tokenizer, pad_on_right, max_length, doc_stride), batched=True, remove_columns=datasets["train"].column_names)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [17]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions'],
        num_rows: 1343
    })
    validation: Dataset({
        features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions'],
        num_rows: 126
    })
})

## Fine-tuning the model

In [18]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [19]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [20]:
from transformers import default_data_collator

data_collator = default_data_collator

In [21]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [22]:
trainer.train()

***** Running training *****
  Num examples = 1343
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 252


Epoch,Training Loss,Validation Loss
1,No log,4.779513
2,No log,4.645035
3,No log,4.368683


***** Running Evaluation *****
  Num examples = 126
  Batch size = 16
***** Running Evaluation *****
  Num examples = 126
  Batch size = 16
***** Running Evaluation *****
  Num examples = 126
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=252, training_loss=3.4457879444909474, metrics={'train_runtime': 268.3111, 'train_samples_per_second': 15.016, 'train_steps_per_second': 0.939, 'total_flos': 394801005021696.0, 'train_loss': 3.4457879444909474, 'epoch': 3.0})

In [23]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
  
tokenizer = AutoTokenizer.from_pretrained("mvonwyl/distilbert-base-uncased-finetuned-squad2")

model = AutoModelForQuestionAnswering.from_pretrained("mvonwyl/distilbert-base-uncased-finetuned-squad2")

https://huggingface.co/mvonwyl/distilbert-base-uncased-finetuned-squad2/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmplrh_dz0g


Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

storing https://huggingface.co/mvonwyl/distilbert-base-uncased-finetuned-squad2/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/0eefcbdecb44adc8a8a31203c6fc6081e4abee730e80b3d7c73ae8149d76268e.42154c5fd30bfa7e34941d0d8ad26f8a3936990926fbe06b2da76dd749b1c6d4
creating metadata file for /root/.cache/huggingface/transformers/0eefcbdecb44adc8a8a31203c6fc6081e4abee730e80b3d7c73ae8149d76268e.42154c5fd30bfa7e34941d0d8ad26f8a3936990926fbe06b2da76dd749b1c6d4
https://huggingface.co/mvonwyl/distilbert-base-uncased-finetuned-squad2/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpmeega4ss


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

storing https://huggingface.co/mvonwyl/distilbert-base-uncased-finetuned-squad2/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/571aebbe8ed96147d4b98134c0a95beec9029368b0c35a1190f40094ee186478.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/571aebbe8ed96147d4b98134c0a95beec9029368b0c35a1190f40094ee186478.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/mvonwyl/distilbert-base-uncased-finetuned-squad2/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp_sqv2dhm


...

storing https://huggingface.co/mvonwyl/distilbert-base-uncased-finetuned-squad2/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/8059125746dc8b7c4ab020d7078038ce3ba711f8ee2be055be536ac398300fe1.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /root/.cache/huggingface/transformers/8059125746dc8b7c4ab020d7078038ce3ba711f8ee2be055be536ac398300fe1.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/mvonwyl/distilbert-base-uncased-finetuned-squad2/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/571aebbe8ed96147d4b98134c0a95beec9029368b0c35a1190f40094ee186478.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/mvonwyl/distilbert-base-uncased-finetuned-squad2/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/28675318b6e2e081432f0e2fa9539dd176e7765623809f220b03c

Downloading:   0%|          | 0.00/561 [00:00<?, ?B/s]

storing https://huggingface.co/mvonwyl/distilbert-base-uncased-finetuned-squad2/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/cffbca5d7cd141d50b708ec6a05c79978d19c4dd3074fcd116c149cd355221f8.a540da103e8b3d40c8db787d1fe802d9215f260969fa27deaa13b88795d8181b
creating metadata file for /root/.cache/huggingface/transformers/cffbca5d7cd141d50b708ec6a05c79978d19c4dd3074fcd116c149cd355221f8.a540da103e8b3d40c8db787d1fe802d9215f260969fa27deaa13b88795d8181b
loading configuration file https://huggingface.co/mvonwyl/distilbert-base-uncased-finetuned-squad2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/cffbca5d7cd141d50b708ec6a05c79978d19c4dd3074fcd116c149cd355221f8.a540da103e8b3d40c8db787d1fe802d9215f260969fa27deaa13b88795d8181b
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "drop

Downloading:   0%|          | 0.00/253M [00:00<?, ?B/s]

storing https://huggingface.co/mvonwyl/distilbert-base-uncased-finetuned-squad2/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/b5b35581fa082df7d2ff2451f726fd2ebd8be192e2a873d4f7e53285f94db045.19e0e53a6167116b32114f34d67c198cfdaa42db408fe90a32add525213a6fc1
creating metadata file for /root/.cache/huggingface/transformers/b5b35581fa082df7d2ff2451f726fd2ebd8be192e2a873d4f7e53285f94db045.19e0e53a6167116b32114f34d67c198cfdaa42db408fe90a32add525213a6fc1
loading weights file https://huggingface.co/mvonwyl/distilbert-base-uncased-finetuned-squad2/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/b5b35581fa082df7d2ff2451f726fd2ebd8be192e2a873d4f7e53285f94db045.19e0e53a6167116b32114f34d67c198cfdaa42db408fe90a32add525213a6fc1
All model checkpoint weights were used when initializing DistilBertForQuestionAnswering.

All the weights of DistilBertForQuestionAnswering were initialized from the model checkpoint at mvonwyl/distilbert-bas

In [24]:
trainer_fine_tuned = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Evaluation

In [25]:
import torch
import numpy as np
import collections

In [26]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [27]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [28]:
type(model)

transformers.models.distilbert.modeling_distilbert.DistilBertForQuestionAnswering

In [29]:
raw_predictions = trainer.predict(validation_features)
raw_predictions_fine_tuned = trainer_fine_tuned.predict(validation_features)

The following columns in the test set  don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id.
***** Running Prediction *****
  Num examples = 126
  Batch size = 16


The following columns in the test set  don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id.
***** Running Prediction *****
  Num examples = 126
  Batch size = 16


In [30]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [31]:
from tqdm.auto import tqdm
import numpy as np

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [32]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)
final_predictions_fine_tuned = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions_fine_tuned.predictions)

Post-processing 119 example predictions split into 126 features.


  0%|          | 0/119 [00:00<?, ?it/s]

Post-processing 119 example predictions split into 126 features.


  0%|          | 0/119 [00:00<?, ?it/s]

In [33]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

Downloading:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

In [34]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'HasAns_exact': 22.641509433962263,
 'HasAns_f1': 28.733289771025618,
 'HasAns_total': 53,
 'NoAns_exact': 0.0,
 'NoAns_f1': 0.0,
 'NoAns_total': 66,
 'best_exact': 56.30252100840336,
 'best_exact_thresh': 0.0,
 'best_f1': 56.30252100840336,
 'best_f1_thresh': 0.0,
 'exact': 10.084033613445378,
 'f1': 12.797179477851746,
 'total': 119}

In [35]:
if squad_v2:
    formatted_predictions_fine_tuned = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions_fine_tuned.items()]
else:
    formatted_predictions_fine_tuned = [{"id": k, "prediction_text": v} for k, v in final_predictions_fine_tuned.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions_fine_tuned, references=references)

{'HasAns_exact': 71.69811320754717,
 'HasAns_f1': 79.64959568733154,
 'HasAns_total': 53,
 'NoAns_exact': 28.78787878787879,
 'NoAns_f1': 28.78787878787879,
 'NoAns_total': 66,
 'best_exact': 59.66386554621849,
 'best_exact_thresh': 0.0,
 'best_f1': 59.66386554621849,
 'best_f1_thresh': 0.0,
 'exact': 47.89915966386555,
 'f1': 51.4405762304922,
 'total': 119}

## Observations
We can clearly see that the distillbert fine tuned model performs better than the default distillbert model, with a difference of f1 score of around 22. Let's dive further to see what caused this difference.

In [36]:
show_first_elements(datasets["validation"], 10)

,id,title,context,question,answers
0,56ddde6b9a695914005b9628,Normans,"The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (""Norman"" comes from ""Norseman"") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.",In what country is Normandy located?,"{'text': ['France', 'France', 'France', 'France'], 'answer_start': [159, 159, 159, 159]}"
1,56ddde6b9a695914005b9629,Normans,"The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (""Norman"" comes from ""Norseman"") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.",When were the Normans in Normandy?,"{'text': ['10th and 11th centuries', 'in the 10th and 11th centuries', '10th and 11th centuries', '10th and 11th centuries'], 'answer_start': [94, 87, 94, 94]}"
2,56ddde6b9a695914005b962a,Normans,"The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (""Norman"" comes from ""Norseman"") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.",From which countries did the Norse originate?,"{'text': ['Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Iceland and Norway'], 'answer_start': [256, 256, 256, 256]}"
3,56ddde6b9a695914005b962b,Normans,"The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (""Norman"" comes from ""Norseman"") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.",Who was the Norse leader?,"{'text': ['Rollo', 'Rollo', 'Rollo', 'Rollo'], 'answer_start': [308, 308, 308, 308]}"
4,56ddde6b9a695914005b962c,Normans,"The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name 

In [37]:
datasets["validation"][5]

{'answers': {'answer_start': [], 'text': []},
 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'id': '5ad39d53604f3c001a3fe8d1',
 'question': "Who gave their name to Normandy in the 1000's and 1100's",
 'title': 'Normans'}

if we take a close look at the 5th row in the validation dataset, we can clearly see that the answer is not the present

In [38]:
formatted_predictions_fine_tuned[5]

{'id': '5ad39d53604f3c001a3fe8d1',
 'no_answer_probability': 0.0,
 'prediction_text': 'The Normans'}

In [39]:
formatted_predictions[5]

{'id': '5ad39d53604f3c001a3fe8d1',
 'no_answer_probability': 0.0,
 'prediction_text': '10th and 11th centuries'}

In fact, the default model didnt pick up the answer as the validation set. But the fine tuned model, which was smarter at finding the answer wasnt gonna get it correctly because the validation example is also empty. That is maybe due to the ambiguity in the question which specified years (validation question) instead of centuries (validation example). But as humans we can easily figure out that the correct answer should indeed be 'Normans'.

In [40]:
n_empty_answers = len([v for v in datasets["validation"] if v["answers"]["text"] == []])
n_empty_answers

66

In [41]:
print(int(n_empty_answers / len(datasets["validation"]) * 100), "%")

55 %


Now we can see that almost 50% of the validation dataset contains no answers. And since the model is sometimes able to predict complexe questions, it is bound to be a source of error in the metrics computation which may seem to lower the performance of the model at first sight

In [43]:
show_gt_vs_predicted(datasets["validation"], 20, formatted_predictions)

,id,title,context,question,answers,predicted
0,56ddde6b9a695914005b9628,Normans,"The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (""Norman"" comes from ""Norseman"") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.",In what country is Normandy located?,"{'text': ['France', 'France', 'France', 'France'], 'answer_start': [159, 159, 159, 159]}","{'id': '56ddde6b9a695914005b9628', 'prediction_text': 'Nourmands; French: Normands; Latin: Normanni', 'no_answer_probability': 0.0}"
1,56ddde6b9a695914005b9629,Normans,"The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (""Norman"" comes from ""Norseman"") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.",When were the Normans in Normandy?,"{'text': ['10th and 11th centuries', 'in the 10th and 11th centuries', '10th and 11th centuries', '10th and 11th centuries'], 'answer_start': [94, 87, 94, 94]}","{'id': '56ddde6b9a695914005b9629', 'prediction_text': 'Nourmands; French: Normands; Latin: Normanni', 'no_answer_probability': 0.0}"
2,56ddde6b9a695914005b962a,Normans,"The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (""Norman"" comes from ""Norseman"") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.",From which countries did the Norse originate?,"{'text': ['Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Iceland and Norway'], 'answer_start': [256, 256, 256, 256]}","{'id': '56ddde6b9a695914005b962a', 'prediction_text': 'Nourmands; French: Normands; Latin: Normanni', 'no_answer_probability': 0.0}"
3,56ddde6b9a695914005b962b,Normans,"The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (""Norman"" comes from ""Norseman"") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of

In [44]:
datasets["validation"][0]["answers"]["text"]

['France', 'France', 'France', 'France']

In [45]:
i = 0
while(formatted_predictions[i]["prediction_text"] in datasets["validation"][i]["answers"]["text"] or datasets["validation"][i]["answers"]["text"] == []):
  i +=1

In [46]:
formatted_predictions[i]["prediction_text"]

'Nourmands; French: Normands; Latin: Normanni'

In [47]:
datasets["validation"][i]["answers"]["text"]

['France', 'France', 'France', 'France']

Here we also see an error caused by the fact that the model took the whole sentence containing the answer, instead of the just capturing the part of text that truly holds the answer, and therefore the noise in the model's answer is in fact an error since it does not corresponds with the validation set's answer.